In [ ]:
  import requests #importing the libraries needed
  from bs4 import BeautifulSoup
  import time
  import json
  import re
  from google.colab import files



  def scrape_poem(url):
    response=requests.get(url)
    poem_lines=[]
    var={"poem":[]}
    if response.status_code==200:
      soup=BeautifulSoup(response.content,'lxml')
      poem=soup.find('div',class_="field field--body")
      if not poem:
        return var
      pom=poem.find_all('p')
      for po in pom:
        ex=po.find_all('span',class_="long-line")
        if(ex):
          for line in ex:
            poem_lines.append(line.text.strip())
          poem_lines.append("")
          var["poem"] = "\n".join(poem_lines).rstrip('\n')
    import html

# After scraping the poem content
    var["poem"] = html.unescape(var["poem"])
    return var




  def scrape_poems(url,list):
      response = requests.get(url)
      if response.status_code == 200: # means the request for html text was succesful
          soup = BeautifulSoup(response.content, 'lxml')
          poems = []
          rows = soup.find_all('tr') #headers for each poem
          if not rows:
              print("No articles found with the specified class.")
              return -1
          for row in rows:
              title_cell = row.find('td', class_="views-field views-field-title") # finding headers for title
              if title_cell:
                #text from the title cell basically includes the title
                  poem_url = "https://poets.org" + title_cell.a['href']
                  var=scrape_poem(poem_url)
                  if(var["poem"]): #href was used for storing url of the poem
                   list.append(var)

              # Uncomment the line below during debugging if needed
              # time.sleep(1)
      else:
          print(f"Failed to retrieve page: {response.status_code}")

  def clean_dataset(list):
    for entry in list:
      poem = entry.get("poem", "").strip()
      # Normalize excessive newlines (keep two for paragraph breaks)
      poem = re.sub(r'\n\n+', '\n\n', poem)  # Replace 3+ newlines with 2

    # Replace common special characters
      poem = poem.replace('’', "'").replace('“', '"').replace('”', '"').replace('—', '-')

    # Remove unwanted symbols (e.g., emojis, rare characters)
      poem = re.sub(r'[^\w\s.,!?\'\"-]','',poem)

      entry["poem"] = poem
    return list


  list=[]
  for i in range(0,200) : #iterate through 10 webpages (for eg)
   scrape_poems(f'https://poets.org/poems?page={i}',list)

  list = clean_dataset(list)

  with open("outfile.json", "w", encoding="utf-8") as f:
      json.dump(list, f,ensure_ascii=False, indent=4)
  files.download('outfile.json')  # Downloads the JSON file to your system



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json
import re
with open('/content/outfile.json', 'r') as f:
  poems = json.load(f)
print(len(poems))


short_poems = []
for entry in poems:
  poem = entry.get("poem", "").strip()
  #print(poem)
  if len(poem.split()) < 700:
    short_poems.append(entry)
    print(poem.split())


with open("Poems_dataset.json", "w", encoding="utf-8") as f:
  json.dump(short_poems, f,ensure_ascii=False, indent=4)
  files.download('Poems_dataset.json')


2901


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>